<a href="https://colab.research.google.com/github/ChyYasir/Imp_tools_and_scripts/blob/main/Summary_From_Youtube_Video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Install Necessary tools and libraries**

In [ ]:
!pip install yt-dlp
!pip install git+https://github.com/openai/whisper.git
!pip install torch
!pip install transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.3/174.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 54.3 MB/s eta 0:00:00
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-egc0czl7
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-egc0czl7
  Resolved https://github.com/openai/whisper.git to commit c0d2f624c09dc18e709e37c2ad90c039a4eb72a2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━

# **Download Audio from YouTube**

> ***Provide a youtube video url***



In [ ]:
import yt_dlp
import os

def download_audio(youtube_url, output_path="audio"):
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': f'{output_path}.%(ext)s',  # No double .mp3
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'quiet': True,
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([youtube_url])

    return f"{output_path}.mp3"  # actual saved path

# Example:
youtube_url = "https://www.youtube.com/watch?v=rIrNIzy6U_g"
audio_path = download_audio(youtube_url)
print(f"Downloaded audio to: {audio_path}")


Downloaded audio to: audio.mp3


# **Transcribe Audio with Whisper**

In [ ]:
import whisper

model = whisper.load_model("base")  # or "small", "medium", "large" for more accuracy
result = model.transcribe("audio.mp3")

transcription = result['text']
print(transcription[:1000])  # preview the first part


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 Welcome to Docker 101. If your goal is to ship software in the real world, one of the most powerful concepts to understand is Containerization when developing locally it solves the age-old problem of it works on my machine and when deploying in the cloud It solves the age-old problem of this architecture doesn't scale over the next few minutes We'll unlock the power inside this container by learning 101 different concepts in terms related to computer science the cloud and of course Stalker I'm guessing you know what a computer is right? It's a box that has three important components inside a CPU for calculating things Random access memory for the applications you're using right now and a disc to store things you might use later This is bare metal hardware but in order to use it we need an operating system Most importantly, the OS provides a kernel that sits on top of the bare metal allowing software applications to run on it in the Olden days you would go to the store and buy software

# **Option A: Using Hugging Face Transformers (free)**

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
chunks = [transcription[i:i+1000] for i in range(0, len(transcription), 1000)]

summary = ""
for chunk in chunks:
    summary_part = summarizer(chunk, max_length=130, min_length=30, do_sample=False)[0]['summary_text']
    summary += summary_part + "\n"

print(summary)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu
Your max_length is set to 130, but your input_length is only 48. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=24)


Learn 101 different concepts in terms related to computer science the cloud and of course Stalker. We'll unlock the power inside this container by learning 101 different Concepts in terms of computer science.
Most software is delivered via the internet through the magic of networking when you watch a YouTube video. Your computer is called a client but you and billions of other users are getting that data from remote computers is called servers. When an app starts reaching Millions of people weird things begin to happen the CPU becomes exhausted handling all the incoming requests Disguise slows down network bandwidth gets maxed out and the database becomes too large to query effectively.
Docker is an open source operating system that lets you run multiple operating systems on a single machine. Applications running on top of the Docker engine all share the same host operating system kernel and use resources dynamically based on their needs.
Docker works in three easy steps first you star